In [2]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import glob
%matplotlib inline

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [5]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [6]:
tracklist = sorted(glob.glob(path+'tracks_nio/*2020*'))

In [166]:
maindf=pd.read_csv(output+'inner/inner_nio_all_amphan.csv',delimiter=r",")
maindf.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf=maindf.drop('a', axis=1)

In [100]:
pdf_amphan=pd.read_csv(target+'2020_pmin_nio_amphan.csv',delimiter=r",")
pdf_amphan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
pdf_amphan=pdf_amphan.drop('a', axis=1)

In [33]:
maindf.shape

(51, 189)

In [7]:
extra=pd.read_csv(output+'nio/outder_2020_nio_amphan.csv',delimiter=r",")

In [8]:
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

In [12]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20200514.csv']

In [10]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2020.nc',datapath+'/rhum/rhum_2020.nc'])

In [11]:
#ssingle level variables
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm1.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [13]:
###
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCs_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outu10','outv10','out2mdewtmp','out2mtmp','outconv_ppt','outtot_cld_ice',\
           'outtot_cldwtr','outtot_cld_rain','outvi_div_cld_froz_wtr','outvi_div_cld_liq_wtr','outvi_div_gpot_flux',\
           'outvi_div_ke_flux','outvi_div_mass_flux','outvi_div_moisture_flux','outvi_div_olr_flux','outvi_div_tot_enrgy_flux',\
           'outvi_ke','outvi_pe_inte','outvi_pe_ie_latentenrgy','outvi_temp','outvi_olr','outvi_tot_enrgy','outvi_moisture_div']
   
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCs_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [125]:
###radiation variables
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outcape','outinst_10m_wnd_gst','outinst_moisture_flux','outinst_ssh_flux','outsurfmean_swr_flux','outsurfmean_lhf',\
           'outsurfmean_lwr_flux','outsurfmean_shf','outdwnwrdmean_swr_flux','outtopmean_lwr_flux','outtopmean_swr_flux',\
           'outvimean_moisture_div','outsurf_lhf','outsurf_shf','outtot_suprcool_liqwtr','outtot_wtr_vpr']
    
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
#########precip, mslp, sst variables############
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2020.nc',datapath+'/mslp/mslp_2020.nc',datapath+'/sst/sst_2020.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2020)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outmslp','outconv_rrate','outls_rrate','outmn_conv_prate',\
                 'outmn_ls_prate','outmn_tot_prate','outsst']
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [124]:
rlev_amphan=pd.DataFrame.from_dict(TCr_ts[0])

{'outu10': [796.2118489583333,
  795.3604166666667,
  811.35234375,
  842.32578125,
  737.3610677083333,
  851.7413411458333,
  967.3741536458333,
  1107.1498697916666,
  826.8921549479167,
  860.7341796875,
  923.772265625,
  961.077734375,
  814.6901692708333,
  977.0291015625,
  1185.5,
  1240.62890625,
  1049.661328125,
  1104.7577473958333,
  1218.8974283854166,
  1341.68720703125,
  1129.2133463541666,
  1232.91396484375,
  1434.5651041666667,
  1571.26474609375,
  1255.4114583333333,
  1234.8201822916667,
  1203.7085286458334,
  1305.4219075520834,
  933.7805989583334,
  1039.9405924479167,
  1233.718359375,
  1365.22119140625,
  1102.44765625,
  1146.0440755208333,
  1214.97177734375,
  1303.9566731770833,
  1079.79912109375,
  1249.2208333333333,
  1411.0537434895834,
  1455.8790364583333,
  1218.5537434895834,
  1258.5065104166667,
  1261.0509440104167,
  1246.373828125,
  1293.8224283854167,
  1233.570703125,
  1325.3516927083333,
  1338.2823893229167,
  1158.1087890625,
  1

In [137]:
var_names=rlev_amphan.columns.values.tolist()

In [138]:
var_names

['outcape',
 'outinst_10m_wnd_gst',
 'outinst_moisture_flux',
 'outinst_ssh_flux',
 'outsurfmean_swr_flux',
 'outsurfmean_lhf',
 'outsurfmean_lwr_flux',
 'outsurfmean_shf',
 'outdwnwrdmean_swr_flux',
 'outtopmean_lwr_flux',
 'outtopmean_swr_flux',
 'outvimean_moisture_div',
 'outsurf_lhf',
 'outsurf_shf',
 'outtot_suprcool_liqwtr',
 'outtot_wtr_vpr']

In [131]:
rlev_amphan=pd.DataFrame.from_dict(TCr_ts[0])

In [132]:
prlev_amphan=pd.DataFrame.from_dict(TCpr_ts[0])

In [133]:
slev_amphan=pd.DataFrame.from_dict(TCs_ts[0])

In [134]:
prlev_amphan['outmslp']=prlev_amphan['outmslp']/100

In [167]:
ts1_amphan=pd.concat([maindf,extra,slev_amphan,rlev_amphan,prlev_amphan], axis=1, join='inner')

In [168]:
ts1_amphan

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,outsurf_shf,outtot_suprcool_liqwtr,outtot_wtr_vpr,outmslp,outconv_rrate,outls_rrate,outmn_conv_prate,outmn_ls_prate,outmn_tot_prate,outsst
0,9.443977,0.000017,1.479861e-05,-2.055685e-07,2.044521e-08,-3.208660e-06,-1.423307e-05,339.279724,399.788208,399.788208,...,-52857.016667,0.045086,56.919672,1007.106956,0.000122,0.000064,0.000120,0.000061,0.000181,303.780343
1,10.779417,0.000017,1.173758e-05,-2.727681e-06,6.448522e-07,-4.403502e-06,-8.983649e-06,338.455536,399.590576,399.590576,...,-90072.333333,0.039300,56.275600,1009.091669,0.000105,0.000069,0.000107,0.000069,0.000177,303.806728
2,9.818153,0.000019,2.675203e-05,-2.884809e-06,-5.460012e-06,-2.467067e-06,-2.088937e-06,333.006073,399.812836,399.812836,...,-147020.483333,0.037482,56.899317,1008.434867,0.000113,0.000059,0.000113,0.000062,0.000175,303.797264
3,10.428172,0.000014,3.031001e-05,1.312114e-05,2.860153e-06,-8.945262e-06,-4.135946e-06,332.727722,401.017426,401.017426,...,-133863.450000,0.022512,56.387941,1005.955230,0.000079,0.000030,0.000067,0.000031,0.000098,303.808816
4,9.367740,0.000017,8.835563e-06,5.820412e-06,-2.608463e-07,-1.286105e-05,-5.172364e-06,335.642517,401.253967,401.253967,...,-65170.233333,0.033208,56.355050,1005.690230,0.000075,0.000038,0.000084,0.000037,0.000121,303.809957
5,10.743498,0.000018,1.427128e-05,-2.993394e-06,-9.626377e-06,-3.538671e-06,2.022263e-06,334.353516,400.910950,400.910950,...,-39391.850000,0.037977,57.618835,1007.680049,0.000066,0.000044,0.000061,0.000042,0.000103,303.773682
6,9.895834,0.000024,8.855084e-07,-1.028073e-05,-1.417113e-05,3.600389e-06,3.052127e-06,336.957031,400.657043,400.657043,...,-43663.933333,0.036449,56.675857,1007.180741,0.000103,0.000049,0.000091,0.000048,0.000139,303.792330
7,10.042229,0.000027,9.779919e-06,-1.813208e-06,-1.196863e-05,3.883316e-06,5.186017e-06,340.442261,400.204529,400.204529,...,-49074.966667,0.041148,55.802006,1005.260196,0.000095,0.000054,0.000092,0.000051,0.000143,303.813331
8,11.158937,0.000014,6.176521e-06,6.071020e-06,-9.802309e-07,-6.296487e-06,-5.115969e-07,339.808533,400.642822,400.642822,...,-58339.383333,0.049213,56.242964,1005.654069,0.000125,0.000063,0.000120,0.000063,0.000183,303.697932
9,10.533839,0.000021,5.537630e-06,1.371684e-05,1.475103e-06,-1.038732e-05,1.208738e-06,340.208252,400.899292,400.899292,...,-112953.983333,0.054285,56.828931,1007.343218,0.000151,0.000059,0.000142,0.000061,0.000202,303.706412


In [36]:
#Pressure vars - vor hum
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvort','outrhum']
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
dm1=xr.open_dataset(datapath+'/pvor/pvor_2020.nc')

In [40]:
#Pressure vars - potential vorticity
TCpv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outpvor']
    pvarname = ['pvor']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCpv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [69]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2020.nc')

In [63]:
dm1

<xarray.Dataset>
Dimensions:  (time: 2928, lon: 1440, lat: 721, plev: 21)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01 ... 2020-12-31T21:00:00
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * lat      (lat) float64 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * plev     (plev) float64 1e+03 2e+03 3e+03 5e+03 ... 9.5e+04 9.75e+04 1e+05
Data variables:
    var129   (time, plev, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.10 (https://mpimet.mpg.d...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Wed Nov 17 04:28:56 2021: cdo -f nc copy GRIB_files/gpot_20...
    CDO:          Climate Data Operators version 1.9.10 (https://mpimet.mpg.d...

In [70]:
# Pressure vars - geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outgpot']
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
dm1=xr.open_dataset(datapath+'/tlevs/tlevs_2020.nc')

In [46]:
#Pressure vars - temperature
TCtl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outtemp']
    pvarname = ['temp']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCtl_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2020.nc')

In [52]:
#Pressure vars - vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvvel']
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/1 [00:00<?, ?it/s]

In [54]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [56]:
preslv #after thisrun dx

['10',
 '20',
 '30',
 '50',
 '70',
 '100',
 '150',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '850',
 '925',
 '975',
 '1000']

In [57]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2020.nc')

In [58]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [67]:
TCgp_ts[0]

{'outgpot': array([[ 8.02029545e-05,  3.86146984e-05,  2.17666675e-05,
          6.45283726e-06,  6.59280292e-06,  9.11490497e-07,
          1.08210467e-07,  1.09812148e-07,  1.02978993e-07,
          1.54273645e-07,  2.35201333e-07,  2.76161397e-07,
          1.97805154e-07,  2.02268711e-07,  1.55587137e-07,
          1.84166129e-07,  3.78728146e-08,  2.54512997e-08],
        [ 8.19729952e-05,  4.03322658e-05,  2.20877565e-05,
          7.02549587e-06,  6.84837578e-06,  1.07088636e-06,
          1.37402094e-07,  1.34515843e-07,  1.21654254e-07,
          1.61764425e-07,  2.47738960e-07,  3.05441972e-07,
          2.11050307e-07,  2.02094088e-07,  1.63331667e-07,
          1.72068637e-07,  7.32349387e-09, -3.73605872e-08],
        [ 8.27589538e-05,  4.08876940e-05,  2.17644129e-05,
          7.46514997e-06,  6.55741605e-06,  1.23599408e-06,
          1.28909354e-07,  1.33192104e-07,  1.15000815e-07,
          1.49956698e-07,  2.51704720e-07,  3.08214476e-07,
          2.12308320e-07,  

In [71]:

tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
temppvordict = {'outpvor_'+preslv[i]:TCpv_ts[0]['outpvor'][:,i] for i in range(18)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
temptempdict = {'outtemp_'+plev[i]:TCtl_ts[0]['outtemp'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict2 = {**temppvordict,**tempgpotdict}
dict3 = {**temptempdict,**tempvveldict}
combine1={**dict1,**dict2}
alldict = {**combine1,**dict3}

In [159]:
#storm1
tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

In [160]:
plev_amphan=pd.DataFrame.from_dict(alldict1)

In [169]:
ts_amphan=pd.concat([pdf_amphan,ts1_amphan,plev_amphan], axis=1, join='inner')

In [170]:
ts_amphan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Columns: 300 entries, Pmin to outvvel_1000
dtypes: float64(300)
memory usage: 119.7 KB


In [171]:
ts_amphan

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1004.738125,9.443977,0.000017,1.479861e-05,-2.055685e-07,2.044521e-08,-3.208660e-06,-1.423307e-05,339.279724,399.788208,...,-0.103420,-0.107131,-0.101457,-0.087925,-0.068865,-0.057725,-0.013576,0.003414,0.013835,0.019966
1,1006.834375,10.779417,0.000017,1.173758e-05,-2.727681e-06,6.448522e-07,-4.403502e-06,-8.983649e-06,338.455536,399.590576,...,-0.109633,-0.119566,-0.103211,-0.074476,-0.046361,-0.037919,-0.017537,-0.005020,0.002374,0.005824
2,1006.085000,9.818153,0.000019,2.675203e-05,-2.884809e-06,-5.460012e-06,-2.467067e-06,-2.088937e-06,333.006073,399.812836,...,-0.104941,-0.112371,-0.080093,-0.045346,-0.023088,-0.027752,-0.023231,-0.019587,-0.020310,-0.020610
3,1003.889375,10.428172,0.000014,3.031001e-05,1.312114e-05,2.860153e-06,-8.945262e-06,-4.135946e-06,332.727722,401.017426,...,-0.078551,-0.080947,-0.052084,-0.019245,-0.003743,-0.012490,-0.018723,-0.023092,-0.028580,-0.031281
4,1003.748125,9.367740,0.000017,8.835563e-06,5.820412e-06,-2.608463e-07,-1.286105e-05,-5.172364e-06,335.642517,401.253967,...,-0.092049,-0.089230,-0.052309,-0.013491,-0.006392,-0.002875,-0.001923,0.000474,0.000254,0.000106
5,1005.976250,10.743498,0.000018,1.427128e-05,-2.993394e-06,-9.626377e-06,-3.538671e-06,2.022263e-06,334.353516,400.910950,...,-0.090587,-0.090009,-0.049881,-0.009302,0.001579,-0.001296,0.009354,0.005599,0.005626,0.007497
6,1004.872500,9.895834,0.000024,8.855084e-07,-1.028073e-05,-1.417113e-05,3.600389e-06,3.052127e-06,336.957031,400.657043,...,-0.066228,-0.068766,-0.038663,-0.029216,-0.042063,-0.056170,-0.043608,-0.036810,-0.029674,-0.023077
7,1002.634375,10.042229,0.000027,9.779919e-06,-1.813208e-06,-1.196863e-05,3.883316e-06,5.186017e-06,340.442261,400.204529,...,-0.063315,-0.069529,-0.053562,-0.056353,-0.071679,-0.078301,-0.056624,-0.035452,-0.019019,-0.010585
8,1002.736875,11.158937,0.000014,6.176521e-06,6.071020e-06,-9.802309e-07,-6.296487e-06,-5.115969e-07,339.808533,400.642822,...,-0.061266,-0.066233,-0.068925,-0.065660,-0.068440,-0.069048,-0.044433,-0.017438,0.005705,0.017490
9,1005.100000,10.533839,0.000021,5.537630e-06,1.371684e-05,1.475103e-06,-1.038732e-05,1.208738e-06,340.208252,400.899292,...,-0.068485,-0.080406,-0.095971,-0.107663,-0.117279,-0.114081,-0.073739,-0.037990,-0.011733,0.000946


In [165]:
var_names=ts_amphan.columns.values.tolist()
var_names

['Pmin',
 'w10max',
 'div100',
 'div200',
 'div250',
 'div300',
 'div400',
 'div500',
 'eqt1000',
 'eqt200',
 'eqt250',
 'eqt300',
 'eqt400',
 'eqt500',
 'eqt600',
 'eqt700',
 'eqt850',
 'eqt925',
 'sh850_200',
 'sh850_250',
 'sh925_200',
 'sh925_250',
 'ws10',
 'wstrss10',
 'u10',
 'v10',
 '2mdewtmp',
 '2mtmp',
 'conv_ppt',
 'tot_cld_ice',
 'tot_cldwtr',
 'tot_cld_rain',
 'vi_div_cld_froz_wtr',
 'vi_div_cld_liq_wtr',
 'vi_div_gpot_flux',
 'vi_div_ke_flux',
 'vi_div_mass_flux',
 'vi_div_moisture_flux',
 'vi_div_olr_flux',
 'vi_div_tot_enrgy_flux',
 'vi_ke',
 'vi_pe_inte',
 'vi_pe_ie_latentenrgy',
 'vi_temp',
 'vi_olr',
 'vi_tot_enrgy',
 'vi_moisture_div',
 'cape',
 'inst_10m_wnd_gst',
 'inst_moisture_flux',
 'inst_ssh_flux',
 'surfmean_swr_flux',
 'surfmean_lhf',
 'surfmean_lwr_flux',
 'surfmean_shf',
 'dwnwrdmean_swr_flux',
 'topmean_lwr_flux',
 'topmean_swr_flux',
 'vimean_moisture_div',
 'surf_lhf',
 'surf_shf',
 'tot_suprcool_liqwtr',
 'tot_wtr_vpr',
 'mslp',
 'conv_rrate',
 'ls_rr

In [172]:
df1=ts_amphan
df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)

In [173]:
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)

In [174]:
df1

,Pmin,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,1004.738125,9.443977,0.000017,1.479861e-05,-2.055685e-07,2.044521e-08,-3.208660e-06,-1.423307e-05,339.279724,399.788208,...,-0.103420,-0.107131,-0.101457,-0.087925,-0.068865,-0.057725,-0.013576,0.003414,0.013835,0.019966
1,1006.834375,10.779417,0.000017,1.173758e-05,-2.727681e-06,6.448522e-07,-4.403502e-06,-8.983649e-06,338.455536,399.590576,...,-0.109633,-0.119566,-0.103211,-0.074476,-0.046361,-0.037919,-0.017537,-0.005020,0.002374,0.005824
2,1006.085000,9.818153,0.000019,2.675203e-05,-2.884809e-06,-5.460012e-06,-2.467067e-06,-2.088937e-06,333.006073,399.812836,...,-0.104941,-0.112371,-0.080093,-0.045346,-0.023088,-0.027752,-0.023231,-0.019587,-0.020310,-0.020610
3,1003.889375,10.428172,0.000014,3.031001e-05,1.312114e-05,2.860153e-06,-8.945262e-06,-4.135946e-06,332.727722,401.017426,...,-0.078551,-0.080947,-0.052084,-0.019245,-0.003743,-0.012490,-0.018723,-0.023092,-0.028580,-0.031281
4,1003.748125,9.367740,0.000017,8.835563e-06,5.820412e-06,-2.608463e-07,-1.286105e-05,-5.172364e-06,335.642517,401.253967,...,-0.092049,-0.089230,-0.052309,-0.013491,-0.006392,-0.002875,-0.001923,0.000474,0.000254,0.000106
5,1005.976250,10.743498,0.000018,1.427128e-05,-2.993394e-06,-9.626377e-06,-3.538671e-06,2.022263e-06,334.353516,400.910950,...,-0.090587,-0.090009,-0.049881,-0.009302,0.001579,-0.001296,0.009354,0.005599,0.005626,0.007497
6,1004.872500,9.895834,0.000024,8.855084e-07,-1.028073e-05,-1.417113e-05,3.600389e-06,3.052127e-06,336.957031,400.657043,...,-0.066228,-0.068766,-0.038663,-0.029216,-0.042063,-0.056170,-0.043608,-0.036810,-0.029674,-0.023077
7,1002.634375,10.042229,0.000027,9.779919e-06,-1.813208e-06,-1.196863e-05,3.883316e-06,5.186017e-06,340.442261,400.204529,...,-0.063315,-0.069529,-0.053562,-0.056353,-0.071679,-0.078301,-0.056624,-0.035452,-0.019019,-0.010585
8,1002.736875,11.158937,0.000014,6.176521e-06,6.071020e-06,-9.802309e-07,-6.296487e-06,-5.115969e-07,339.808533,400.642822,...,-0.061266,-0.066233,-0.068925,-0.065660,-0.068440,-0.069048,-0.044433,-0.017438,0.005705,0.017490
9,1005.100000,10.533839,0.000021,5.537630e-06,1.371684e-05,1.475103e-06,-1.038732e-05,1.208738e-06,340.208252,400.899292,...,-0.068485,-0.080406,-0.095971,-0.107663,-0.117279,-0.114081,-0.073739,-0.037990,-0.011733,0.000946


In [175]:
df1.to_csv(output+'timeseries_all_amphan.csv')


In [157]:
var_names=df1.columns.values.tolist()
var_names

['Pmin',
 'w10max',
 'div100',
 'div200',
 'div250',
 'div300',
 'div400',
 'div500',
 'eqt1000',
 'eqt200',
 'eqt250',
 'eqt300',
 'eqt400',
 'eqt500',
 'eqt600',
 'eqt700',
 'eqt850',
 'eqt925',
 'sh850_200',
 'sh850_250',
 'sh925_200',
 'sh925_250',
 'wstrss10',
 '2mdewtmp',
 '2mtmp',
 'conv_ppt',
 'tot_cld_ice',
 'tot_cldwtr',
 'tot_cld_rain',
 'vi_div_cld_froz_wtr',
 'vi_div_cld_liq_wtr',
 'vi_div_mass_flux',
 'vi_div_moisture_flux',
 'vi_div_tot_enrgy_flux',
 'vi_pe_inte',
 'vi_pe_ie_latentenrgy',
 'vi_olr',
 'vi_tot_enrgy',
 'vi_moisture_div',
 'cape',
 'inst_moisture_flux',
 'inst_ssh_flux',
 'surfmean_lhf',
 'surfmean_shf',
 'vimean_moisture_div',
 'surf_lhf',
 'surf_shf',
 'tot_suprcool_liqwtr',
 'tot_wtr_vpr',
 'conv_rrate',
 'ls_rrate',
 'mn_conv_prate',
 'mn_ls_prate',
 'mn_tot_prate',
 'sst',
 'vort_10',
 'vort_20',
 'vort_30',
 'vort_50',
 'vort_70',
 'vort_100',
 'vort_150',
 'vort_200',
 'vort_250',
 'vort_300',
 'vort_400',
 'vort_500',
 'vort_600',
 'vort_700',
 'vor

In [61]:
#NWP
maindf_nisarga=pd.read_csv(target+'2020_nisarga.csv',delimiter=r",")
extra_nisarga=pd.read_csv(target+'outer_derived_nisarga.csv',delimiter=r",")
extra_nisarga.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra_nisarga=extra_nisarga.drop('a', axis=1)

In [62]:
rlev_nisarga=pd.DataFrame.from_dict(TCr_ts[1])
prlev_nisarga=pd.DataFrame.from_dict(TCpr_ts[1])
slev_nisarga=pd.DataFrame.from_dict(TCs_ts[1])
prlev_nisarga['mslp']=prlev_nisarga['mslp']/100

In [63]:
prlev_nisarga['mslp']

0     1010.439049
1     1009.355134
2     1009.114554
3     1010.782933
4     1010.907113
5     1008.866618
6     1007.796767
7     1009.049323
8     1009.878195
9     1007.904021
10    1007.366968
11    1008.851445
12    1008.774283
13    1006.883370
14    1005.650855
15    1006.629798
16    1007.304882
17    1005.592538
18    1005.046042
19    1006.689151
20    1006.849440
21    1004.407252
Name: mslp, dtype: float64

In [70]:
ts1_nisarga=pd.concat([maindf_nisarga,extra_nisarga,slev_nisarga,rlev_nisarga,prlev_nisarga], axis=1, join='inner')

In [71]:
ts1_nisarga

,time,P_min,W10_max,div200,div300,div400,div500,eqt1000,eqt200,eqt250,...,surf_shf,tot_suprcool_liqwtr,tot_wtr_vpr,mslp,conv_rrate,ls_rrate,mn_conv_prate,mn_ls_prate,mn_tot_prate,sst
0,2020-05-31:18.00.00,1009,9,6.549553e-06,3.061710e-07,-1.485475e-06,-2.401858e-07,335.690051,353.439327,351.564334,...,-27833.283333,0.025116,55.176070,1010.439049,0.000052,0.000010,0.000053,0.000010,0.000063,303.360602
1,2020-05-31:21.00.00,1007,10,8.157291e-06,6.724656e-07,1.581929e-08,2.488271e-07,335.367232,352.931890,351.093504,...,-39776.450000,0.041110,55.370038,1009.355134,0.000089,0.000043,0.000081,0.000039,0.000121,303.370086
2,2020-06-01:00.00.00,1007,11,9.613719e-06,9.670616e-08,2.045147e-07,-7.525372e-07,334.226678,352.975657,350.935198,...,-45680.609959,0.045743,55.355389,1009.114554,0.000093,0.000051,0.000098,0.000050,0.000148,303.344453
3,2020-06-01:03.00.00,1007,13,8.478898e-06,-1.238960e-07,1.590575e-06,-6.541119e-07,334.306408,352.948344,350.724187,...,-101064.394191,0.047945,55.083533,1010.782933,0.000076,0.000051,0.000079,0.000050,0.000130,303.351810
4,2020-06-01:06.00.00,1006,15,6.066197e-06,1.428420e-06,4.451718e-06,5.739205e-07,335.077402,353.400239,350.830903,...,-231626.533333,0.053480,55.126486,1010.907113,0.000077,0.000058,0.000076,0.000058,0.000134,303.351109
5,2020-06-01:09.00.00,1004,13,7.184855e-06,4.082880e-06,5.062434e-06,-7.267209e-07,335.999195,353.948273,351.280941,...,-193883.083333,0.058927,54.792557,1008.866618,0.000125,0.000073,0.000117,0.000074,0.000191,303.365956
6,2020-06-01:12.00.00,1003,13,6.812031e-06,4.720119e-06,2.438636e-06,-2.180149e-06,335.618809,354.736426,352.085534,...,-83142.833333,0.056192,53.989836,1007.796767,0.000149,0.000058,0.000152,0.000059,0.000211,303.384655
7,2020-06-01:15.00.00,1003,14,8.477096e-06,5.364593e-06,8.863491e-07,-2.706559e-06,335.334629,354.552624,352.240658,...,-37216.283333,0.042926,53.776609,1009.049323,0.000147,0.000048,0.000153,0.000048,0.000200,303.390974
8,2020-06-01:18.00.00,1003,14,8.844455e-06,3.203878e-06,-5.104724e-07,-2.759216e-06,334.653454,354.217467,352.119715,...,-36337.583333,0.039876,53.334198,1009.878195,0.000097,0.000042,0.000113,0.000043,0.000157,303.395933
9,2020-06-01:21.00.00,1001,14,9.077169e-06,-4.509747e-09,-7.942209e-08,-2.986681e-06,334.497783,353.891898,351.822545,...,-48570.082988,0.059890,52.461350,1007.904021,0.000094,0.000077,0.000092,0.000077,0.000169,303.406193


In [66]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]
tempvortdict2 = {'vort_'+plev[i]:TCp_ts[1]['vort'][:,i] for i in range(21)}
temprhumdict2 = {'rhum_'+plev[i]:TCp_ts[1]['rhum'][:,i] for i in range(21)}
temppvordict2 = {'pvor_'+preslv[i]:TCpv_ts[1]['pvor'][:,i] for i in range(18)}
tempgpotdict2 = {'gpot_'+plev[i]:TCgp_ts[1]['gpot'][:,i] for i in range(21)}
temptempdict2 = {'temp_'+plev[i]:TCtl_ts[1]['temp'][:,i] for i in range(21)}
tempvveldict2 = {'vvel_'+preslv[i]:TCvv_ts[1]['vvel'][:,i] for i in range(18)}
dict21 = {**tempvortdict2,**temprhumdict2}
dict22 = {**temppvordict2,**tempgpotdict2}
dict23 = {**temptempdict2,**tempvveldict2}
combine21={**dict21,**dict22}
alldict2 = {**combine21,**dict23}

In [72]:
plev_nisarga=pd.DataFrame.from_dict(alldict2)
ts_nisarga=pd.concat([ts1_nisarga,plev_nisarga], axis=1, join='inner')

In [73]:
ts_nisarga

,time,P_min,W10_max,div200,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_800,vvel_850,vvel_900,vvel_925
0,2020-05-31:18.00.00,1009,9,6.549553e-06,3.061710e-07,-1.485475e-06,-2.401858e-07,335.690051,353.439327,351.564334,...,-0.064186,-0.069888,-0.060469,-0.045601,-0.046245,-0.055677,-0.045091,-0.028809,-0.014700,-0.006641
1,2020-05-31:21.00.00,1007,10,8.157291e-06,6.724656e-07,1.581929e-08,2.488271e-07,335.367232,352.931890,351.093504,...,-0.074557,-0.087519,-0.085384,-0.084092,-0.086217,-0.092433,-0.065151,-0.045091,-0.024690,-0.012680
2,2020-06-01:00.00.00,1007,11,9.613719e-06,9.670616e-08,2.045147e-07,-7.525372e-07,334.226678,352.975657,350.935198,...,-0.074513,-0.083865,-0.088879,-0.083235,-0.072592,-0.068658,-0.027372,-0.009276,0.005941,0.016104
3,2020-06-01:03.00.00,1007,13,8.478898e-06,-1.238960e-07,1.590575e-06,-6.541119e-07,334.306408,352.948344,350.724187,...,-0.067978,-0.075626,-0.088279,-0.086260,-0.066925,-0.050308,-0.015688,-0.006042,0.003474,0.010849
4,2020-06-01:06.00.00,1006,15,6.066197e-06,1.428420e-06,4.451718e-06,5.739205e-07,335.077402,353.400239,350.830903,...,-0.062146,-0.077541,-0.112650,-0.134434,-0.119242,-0.092720,-0.059834,-0.038594,-0.025939,-0.017274
5,2020-06-01:09.00.00,1004,13,7.184855e-06,4.082880e-06,5.062434e-06,-7.267209e-07,335.999195,353.948273,351.280941,...,-0.082942,-0.109512,-0.159276,-0.173615,-0.156798,-0.123146,-0.082672,-0.054409,-0.038012,-0.028892
6,2020-06-01:12.00.00,1003,13,6.812031e-06,4.720119e-06,2.438636e-06,-2.180149e-06,335.618809,354.736426,352.085534,...,-0.107053,-0.139685,-0.168457,-0.169205,-0.149990,-0.123590,-0.060305,-0.023592,-0.004696,0.001307
7,2020-06-01:15.00.00,1003,14,8.477096e-06,5.364593e-06,8.863491e-07,-2.706559e-06,335.334629,354.552624,352.240658,...,-0.110425,-0.143450,-0.165902,-0.154650,-0.131029,-0.103397,-0.042353,-0.014263,0.004163,0.011290
8,2020-06-01:18.00.00,1003,14,8.844455e-06,3.203878e-06,-5.104724e-07,-2.759216e-06,334.653454,354.217467,352.119715,...,-0.108992,-0.133102,-0.139159,-0.122790,-0.103295,-0.089037,-0.047533,-0.027492,-0.015471,-0.009360
9,2020-06-01:21.00.00,1001,14,9.077169e-06,-4.509747e-09,-7.942209e-08,-2.986681e-06,334.497783,353.891898,351.822545,...,-0.112870,-0.120849,-0.121042,-0.107082,-0.083890,-0.073221,-0.058333,-0.038679,-0.023037,-0.015045


In [74]:
ts_nisarga.to_csv(output+'outer_all_nisarga.csv')

In [75]:
#gati
#Gati t=33

maindf_gati=pd.read_csv(target+'2020_gati.csv',delimiter=r",")
extra_gati=pd.read_csv(target+'outer_derived_gati.csv',delimiter=r",")
extra_gati.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra_gati=extra_gati.drop('a', axis=1)

In [76]:
maindf_gati

,time,P_min,W10_max
0,2020-11-20:12.00.00,1006,11
1,2020-11-20:15.00.00,1008,12
2,2020-11-20:18.00.00,1009,12
3,2020-11-20:21.00.00,1007,12
4,2020-11-21:00.00.00,1006,12
5,2020-11-21:03.00.00,1008,14
6,2020-11-21:06.00.00,1009,14
7,2020-11-21:09.00.00,1006,14
8,2020-11-21:12.00.00,1005,13
9,2020-11-21:15.00.00,1007,13


In [78]:
rlev_gati=pd.DataFrame.from_dict(TCr_ts[2])
prlev_gati=pd.DataFrame.from_dict(TCpr_ts[2])
slev_gati=pd.DataFrame.from_dict(TCs_ts[2])
prlev_gati['mslp']=prlev_gati['mslp']/100

In [79]:
prlev_gati['mslp']

0     1008.950092
1     1010.740368
2     1011.607100
3     1010.091743
4     1009.820469
5     1011.744803
6     1012.867945
7     1010.748440
8     1009.585096
9     1011.349511
10    1012.434626
11    1011.197568
12    1010.945621
13    1012.559944
14    1014.122075
15    1012.150505
16    1010.355064
17    1012.185434
18    1013.890886
19    1012.994936
20    1012.624119
21    1014.106549
22    1015.836383
23    1013.414281
24    1011.226492
25    1012.861041
26    1014.953944
27    1014.164949
28    1013.028285
29    1014.205030
30    1015.482636
31    1012.447001
32    1009.746618
Name: mslp, dtype: float64

In [80]:
ts1_gati=pd.concat([maindf_gati,extra_gati,slev_gati,rlev_gati,prlev_gati], axis=1, join='inner')

In [83]:
plev

['10',
 '20',
 '30',
 '50',
 '70',
 '100',
 '150',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '800',
 '850',
 '900',
 '925',
 '950',
 '975',
 '1000']

In [85]:
preslv

['10',
 '20',
 '30',
 '50',
 '70',
 '100',
 '150',
 '200',
 '250',
 '300',
 '400',
 '500',
 '600',
 '700',
 '800',
 '850',
 '900',
 '925',
 '950',
 '975',
 '1000']

In [86]:
tempvortdict3 = {'vort_'+plev[i]:TCp_ts[2]['vort'][:,i] for i in range(21)}
temprhumdict3 = {'rhum_'+plev[i]:TCp_ts[2]['rhum'][:,i] for i in range(21)}
temppvordict3 = {'pvor_'+preslv[i]:TCpv_ts[2]['pvor'][:,i] for i in range(18)}
tempgpotdict3 = {'gpot_'+plev[i]:TCgp_ts[2]['gpot'][:,i] for i in range(21)}
temptempdict3 = {'temp_'+plev[i]:TCtl_ts[2]['temp'][:,i] for i in range(21)}
tempvveldict3 = {'vvel_'+preslv[i]:TCvv_ts[2]['vvel'][:,i] for i in range(18)}
dict31 = {**tempvortdict3,**temprhumdict3}
dict32 = {**temppvordict3,**tempgpotdict3}
dict33 = {**temptempdict3,**tempvveldict3}
combine31={**dict31,**dict32}
alldict3 = {**combine31,**dict33}

In [91]:
temppvordict3

{'pvor_10': array([4.66630949e-05, 4.64138032e-05, 5.04701631e-05, 5.55053120e-05,
        5.77273158e-05, 5.78754504e-05, 5.66671370e-05, 5.45050095e-05,
        5.32677552e-05, 5.18389512e-05, 5.06119182e-05, 5.03432394e-05,
        4.74360383e-05, 4.41223461e-05, 4.07084978e-05, 3.90536973e-05,
        3.91324037e-05, 4.08809464e-05, 4.34474453e-05, 4.52655368e-05,
        4.64294956e-05, 4.72738563e-05, 5.06747669e-05, 5.38779733e-05,
        5.48076872e-05, 5.65909100e-05, 6.03941996e-05, 5.99817551e-05,
        6.01319053e-05, 5.75416954e-05, 5.64802865e-05, 5.42985974e-05,
        5.00004195e-05]),
 'pvor_20': array([3.98155302e-05, 3.97589543e-05, 4.05279687e-05, 4.17732246e-05,
        4.25239637e-05, 4.36793819e-05, 4.45298657e-05, 4.39674477e-05,
        4.33830467e-05, 4.28460344e-05, 4.29078044e-05, 4.31796963e-05,
        4.16659168e-05, 4.16799487e-05, 4.19892565e-05, 4.20327625e-05,
        4.10550544e-05, 4.13771655e-05, 4.21514742e-05, 4.24428610e-05,
        4.274346

In [93]:
plev_gati=pd.DataFrame.from_dict(alldict3)

In [94]:
plev_gati

,vort_10,vort_20,vort_30,vort_50,vort_70,vort_100,vort_150,vort_200,vort_250,vort_300,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_800,vvel_850,vvel_900,vvel_925
0,-0.000010,0.000006,0.000004,3.348969e-06,3.240374e-06,-8.013103e-06,-0.000022,-0.000021,-0.000020,-0.000015,...,-0.062231,-0.057401,-0.031045,-0.017085,-0.019584,-0.019478,-0.020349,-0.012713,0.001450,0.008742
1,-0.000010,0.000007,0.000004,2.333138e-06,2.521471e-06,-8.065954e-06,-0.000023,-0.000021,-0.000021,-0.000015,...,-0.053229,-0.037502,-0.004269,0.000441,0.003136,0.004744,-0.001887,-0.000406,0.008550,0.013755
2,-0.000009,0.000008,0.000005,8.405147e-07,1.054288e-06,-6.609703e-06,-0.000023,-0.000022,-0.000021,-0.000014,...,-0.048581,-0.030541,-0.008612,-0.016596,-0.010349,-0.010622,-0.029283,-0.031947,-0.024376,-0.018906
3,-0.000008,0.000007,0.000005,-4.294522e-07,3.048384e-07,-5.616709e-06,-0.000023,-0.000022,-0.000021,-0.000014,...,-0.059040,-0.043977,-0.032680,-0.044638,-0.036667,-0.030748,-0.039312,-0.037162,-0.025812,-0.019072
4,-0.000008,0.000005,0.000006,-9.614586e-07,6.655115e-07,-4.063725e-06,-0.000024,-0.000022,-0.000019,-0.000011,...,-0.036508,-0.026365,-0.013801,-0.021418,-0.021380,-0.018359,-0.012088,-0.010494,-0.000215,0.005773
5,-0.000007,0.000003,0.000006,5.975831e-07,1.635000e-06,-2.274686e-06,-0.000022,-0.000022,-0.000019,-0.000011,...,-0.028605,-0.015118,-0.009013,-0.021130,-0.020223,-0.013519,-0.001038,0.001551,0.012972,0.019594
6,-0.000007,0.000003,0.000005,2.394663e-06,1.657324e-06,-2.031680e-07,-0.000020,-0.000021,-0.000018,-0.000011,...,-0.026880,-0.020479,-0.027250,-0.037009,-0.035134,-0.035709,-0.038983,-0.033349,-0.023137,-0.017341
7,-0.000008,0.000003,0.000003,3.659838e-06,4.803422e-07,7.057718e-07,-0.000018,-0.000019,-0.000018,-0.000012,...,-0.025350,-0.025150,-0.028783,-0.022494,-0.009944,-0.010535,-0.034831,-0.036272,-0.037058,-0.038586
8,-0.000010,0.000004,0.000002,4.737141e-06,-7.100753e-08,2.817750e-06,-0.000018,-0.000021,-0.000018,-0.000010,...,-0.033797,-0.041580,-0.024790,-0.002024,0.011485,0.018436,0.011547,0.007605,-0.002369,-0.008707
9,-0.000010,0.000007,0.000002,5.326160e-06,-3.340751e-07,4.456126e-06,-0.000017,-0.000020,-0.000017,-0.000010,...,-0.015731,-0.018101,0.008267,0.027852,0.042333,0.043969,0.037639,0.023248,0.007181,0.000109


In [95]:
ts_gati=pd.concat([ts1_gati,plev_gati], axis=1, join='inner')

In [96]:
ts_gati

,time,P_min,W10_max,div200,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_800,vvel_850,vvel_900,vvel_925
0,2020-11-20:12.00.00,1006,11,0.000005,-1.154400e-06,-3.528275e-06,-3.267141e-07,327.657727,350.393842,348.081267,...,-0.062231,-0.057401,-0.031045,-0.017085,-0.019584,-0.019478,-0.020349,-0.012713,0.001450,0.008742
1,2020-11-20:15.00.00,1008,12,0.000005,-3.671897e-06,-1.922008e-06,-1.739291e-07,327.587241,349.989797,347.607647,...,-0.053229,-0.037502,-0.004269,0.000441,0.003136,0.004744,-0.001887,-0.000406,0.008550,0.013755
2,2020-11-20:18.00.00,1009,12,0.000004,-4.367318e-06,3.888525e-07,2.071516e-07,327.335877,349.567188,347.174773,...,-0.048581,-0.030541,-0.008612,-0.016596,-0.010349,-0.010622,-0.029283,-0.031947,-0.024376,-0.018906
3,2020-11-20:21.00.00,1007,12,0.000005,-3.931311e-06,9.514229e-07,5.436587e-07,326.965015,349.251891,346.896104,...,-0.059040,-0.043977,-0.032680,-0.044638,-0.036667,-0.030748,-0.039312,-0.037162,-0.025812,-0.019072
4,2020-11-21:00.00.00,1006,12,0.000005,-2.795625e-06,-1.920017e-07,1.357307e-06,326.336305,349.340209,346.966220,...,-0.036508,-0.026365,-0.013801,-0.021418,-0.021380,-0.018359,-0.012088,-0.010494,-0.000215,0.005773
5,2020-11-21:03.00.00,1008,14,0.000006,-3.014734e-06,8.809641e-07,1.278106e-06,325.745012,349.204505,346.791980,...,-0.028605,-0.015118,-0.009013,-0.021130,-0.020223,-0.013519,-0.001038,0.001551,0.012972,0.019594
6,2020-11-21:06.00.00,1009,14,0.000005,-1.357792e-06,1.282567e-06,8.017600e-07,325.593103,349.446355,346.993915,...,-0.026880,-0.020479,-0.027250,-0.037009,-0.035134,-0.035709,-0.038983,-0.033349,-0.023137,-0.017341
7,2020-11-21:09.00.00,1006,14,0.000005,-1.937800e-07,-2.018370e-08,-5.707132e-07,325.833844,349.832071,347.289594,...,-0.025350,-0.025150,-0.028783,-0.022494,-0.009944,-0.010535,-0.034831,-0.036272,-0.037058,-0.038586
8,2020-11-21:12.00.00,1005,13,0.000007,4.941363e-08,-2.667999e-06,-9.972503e-07,325.645202,350.249386,347.599469,...,-0.033797,-0.041580,-0.024790,-0.002024,0.011485,0.018436,0.011547,0.007605,-0.002369,-0.008707
9,2020-11-21:15.00.00,1007,13,0.000005,-7.546073e-07,-3.256019e-06,-5.044041e-07,325.178662,349.965892,347.294114,...,-0.015731,-0.018101,0.008267,0.027852,0.042333,0.043969,0.037639,0.023248,0.007181,0.000109


In [97]:
ts_gati.to_csv(output+'outer_all_gati.csv')

In [98]:
#nivar 35
maindf_nivar=pd.read_csv(target+'2020_nivar.csv',delimiter=r",")
extra_nivar=pd.read_csv(target+'outer_derived_nivar.csv',delimiter=r",")
extra_nivar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra_nivar=extra_nivar.drop('a', axis=1)

In [99]:
rlev_nivar=pd.DataFrame.from_dict(TCr_ts[3])
prlev_nivar=pd.DataFrame.from_dict(TCpr_ts[3])
slev_nivar=pd.DataFrame.from_dict(TCs_ts[3])
prlev_nivar['mslp']=prlev_nivar['mslp']/100

In [100]:
ts1_nivar=pd.concat([maindf_nivar,extra_nivar,slev_nivar,rlev_nivar,prlev_nivar], axis=1, join='inner')

In [101]:
ts1_nivar

,time,P_min,W10_max,div200,div300,div400,div500,eqt1000,eqt200,eqt250,...,surf_shf,tot_suprcool_liqwtr,tot_wtr_vpr,mslp,conv_rrate,ls_rrate,mn_conv_prate,mn_ls_prate,mn_tot_prate,sst
0,2020-11-22:06.00.00,1008,10,0.000007,3.401243e-06,6.405327e-07,-3.814991e-07,327.948702,349.371367,347.585886,...,-74595.466667,0.057252,52.886633,1011.763693,0.000074,0.000066,0.000071,0.000060,0.000131,302.179608
1,2020-11-22:09.00.00,1006,12,0.000008,3.235708e-06,6.048645e-07,-1.547353e-06,328.661733,349.639196,347.945505,...,-69334.366667,0.045305,52.512545,1009.502680,0.000078,0.000056,0.000076,0.000056,0.000132,302.178821
2,2020-11-22:12.00.00,1008,9,0.000013,1.618296e-06,-1.745275e-06,-3.729949e-06,327.894637,349.921767,348.284484,...,-36736.323651,0.032639,51.618280,1010.963493,0.000106,0.000044,0.000108,0.000046,0.000154,302.172648
3,2020-11-22:15.00.00,1010,10,0.000013,4.282262e-07,-2.447906e-06,-3.441541e-06,327.637625,349.447333,347.952487,...,-36087.566667,0.033529,51.263432,1013.238594,0.000098,0.000041,0.000098,0.000039,0.000137,302.170890
4,2020-11-22:18.00.00,1009,11,0.000014,4.532094e-07,-3.271727e-06,-3.428423e-06,327.627211,348.934070,347.656981,...,-42215.734440,0.035596,50.730315,1012.643459,0.000104,0.000050,0.000103,0.000048,0.000151,302.167236
5,2020-11-22:21.00.00,1007,12,0.000014,1.178843e-06,-3.472468e-06,-5.514498e-06,327.909836,348.553285,347.429604,...,-46761.091667,0.033449,50.914341,1010.737171,0.000058,0.000047,0.000062,0.000046,0.000108,302.166191
6,2020-11-23:00.00.00,1006,13,0.000014,5.274590e-07,-2.418490e-06,-4.135727e-06,326.838229,348.670526,347.279576,...,-43950.004149,0.032494,49.257880,1011.061491,0.000053,0.000043,0.000055,0.000040,0.000095,302.151790
7,2020-11-23:03.00.00,1008,13,0.000013,1.033984e-07,-1.397790e-06,-2.750894e-06,326.928653,348.637400,347.235834,...,-60977.398340,0.032174,48.981777,1013.545690,0.000064,0.000042,0.000062,0.000042,0.000104,302.142322
8,2020-11-23:06.00.00,1007,13,0.000013,1.962774e-06,2.047650e-07,-2.571119e-06,327.727050,349.102494,347.594306,...,-132879.207469,0.035925,49.020348,1012.326543,0.000073,0.000044,0.000072,0.000042,0.000113,302.140731
9,2020-11-23:09.00.00,1004,13,0.000014,6.640247e-07,-2.138302e-06,-2.802736e-06,328.500958,349.547546,347.932249,...,-103537.633333,0.038125,49.262877,1009.860682,0.000057,0.000053,0.000060,0.000055,0.000115,302.139604


In [102]:
tempvortdict4 = {'vort_'+plev[i]:TCp_ts[3]['vort'][:,i] for i in range(21)}
temprhumdict4 = {'rhum_'+plev[i]:TCp_ts[3]['rhum'][:,i] for i in range(21)}
temppvordict4 = {'pvor_'+preslv[i]:TCpv_ts[3]['pvor'][:,i] for i in range(18)}
tempgpotdict4 = {'gpot_'+plev[i]:TCgp_ts[3]['gpot'][:,i] for i in range(21)}
temptempdict4 = {'temp_'+plev[i]:TCtl_ts[3]['temp'][:,i] for i in range(21)}
tempvveldict4 = {'vvel_'+preslv[i]:TCvv_ts[3]['vvel'][:,i] for i in range(18)}
dict41 = {**tempvortdict4,**temprhumdict4}
dict42 = {**temppvordict4,**tempgpotdict4}
dict43 = {**temptempdict4,**tempvveldict4}
combine41={**dict41,**dict42}
alldict4 = {**combine41,**dict43}

In [103]:
plev_nivar=pd.DataFrame.from_dict(alldict4)

In [104]:
plev_nivar

,vort_10,vort_20,vort_30,vort_50,vort_70,vort_100,vort_150,vort_200,vort_250,vort_300,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_800,vvel_850,vvel_900,vvel_925
0,-0.000007,5.475822e-06,0.000006,2.976719e-06,5.085117e-06,-3.231588e-06,-0.000017,-0.000015,-0.000013,-0.000008,...,-0.066463,-0.089895,-0.109808,-0.108040,-0.088148,-0.077671,-0.056594,-0.048633,-0.038558,-0.032745
1,-0.000006,4.630011e-06,0.000007,1.949822e-06,4.891615e-06,-2.293792e-06,-0.000017,-0.000017,-0.000013,-0.000008,...,-0.078617,-0.100976,-0.109420,-0.105475,-0.080158,-0.065640,-0.040874,-0.029021,-0.019763,-0.014881
2,-0.000005,3.661937e-06,0.000008,2.344465e-06,3.956531e-06,7.096639e-07,-0.000018,-0.000017,-0.000015,-0.000011,...,-0.132909,-0.155783,-0.143317,-0.124282,-0.078284,-0.044831,0.001800,0.011558,0.017232,0.020754
3,-0.000005,2.700424e-06,0.000008,3.458233e-06,3.125358e-06,-3.677085e-07,-0.000017,-0.000017,-0.000016,-0.000012,...,-0.123128,-0.140306,-0.126094,-0.105192,-0.061462,-0.028039,0.006609,0.006726,0.007293,0.008588
4,-0.000005,1.165609e-06,0.000007,3.876758e-06,2.900404e-06,-7.105238e-07,-0.000018,-0.000018,-0.000015,-0.000012,...,-0.110378,-0.123931,-0.108033,-0.073024,-0.036060,-0.014356,-0.008103,-0.015680,-0.017296,-0.016317
5,-0.000006,-7.372738e-07,0.000007,4.125063e-06,2.161340e-06,-3.558875e-07,-0.000019,-0.000017,-0.000015,-0.000011,...,-0.097499,-0.115552,-0.103627,-0.055830,-0.008271,0.009142,-0.004370,-0.011956,-0.012377,-0.010081
6,-0.000008,-5.191041e-07,0.000005,4.100884e-06,1.090100e-06,-8.419417e-08,-0.000019,-0.000016,-0.000015,-0.000010,...,-0.096495,-0.110931,-0.098452,-0.071045,-0.031172,-0.010264,-0.008990,-0.002605,0.009843,0.017592
7,-0.000009,-2.409981e-07,0.000005,4.010283e-06,7.975929e-07,1.582398e-06,-0.000021,-0.000018,-0.000015,-0.000010,...,-0.084414,-0.093452,-0.087313,-0.070638,-0.038386,-0.018827,-0.012818,-0.005613,0.005607,0.011993
8,-0.000010,5.648269e-07,0.000004,3.156576e-06,1.604373e-06,2.568871e-06,-0.000020,-0.000017,-0.000014,-0.000009,...,-0.092110,-0.107825,-0.124465,-0.110838,-0.082611,-0.060366,-0.051719,-0.045475,-0.038422,-0.033897
9,-0.000011,3.714473e-07,0.000004,1.732755e-06,2.409621e-06,2.666679e-06,-0.000019,-0.000019,-0.000015,-0.000009,...,-0.102938,-0.112898,-0.110809,-0.082311,-0.057175,-0.035954,-0.037499,-0.033784,-0.026006,-0.020837


In [105]:
ts_nivar=pd.concat([ts1_nivar,plev_nivar], axis=1, join='inner')

In [106]:
ts_nivar

,time,P_min,W10_max,div200,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_800,vvel_850,vvel_900,vvel_925
0,2020-11-22:06.00.00,1008,10,0.000007,3.401243e-06,6.405327e-07,-3.814991e-07,327.948702,349.371367,347.585886,...,-0.066463,-0.089895,-0.109808,-0.108040,-0.088148,-0.077671,-0.056594,-0.048633,-0.038558,-0.032745
1,2020-11-22:09.00.00,1006,12,0.000008,3.235708e-06,6.048645e-07,-1.547353e-06,328.661733,349.639196,347.945505,...,-0.078617,-0.100976,-0.109420,-0.105475,-0.080158,-0.065640,-0.040874,-0.029021,-0.019763,-0.014881
2,2020-11-22:12.00.00,1008,9,0.000013,1.618296e-06,-1.745275e-06,-3.729949e-06,327.894637,349.921767,348.284484,...,-0.132909,-0.155783,-0.143317,-0.124282,-0.078284,-0.044831,0.001800,0.011558,0.017232,0.020754
3,2020-11-22:15.00.00,1010,10,0.000013,4.282262e-07,-2.447906e-06,-3.441541e-06,327.637625,349.447333,347.952487,...,-0.123128,-0.140306,-0.126094,-0.105192,-0.061462,-0.028039,0.006609,0.006726,0.007293,0.008588
4,2020-11-22:18.00.00,1009,11,0.000014,4.532094e-07,-3.271727e-06,-3.428423e-06,327.627211,348.934070,347.656981,...,-0.110378,-0.123931,-0.108033,-0.073024,-0.036060,-0.014356,-0.008103,-0.015680,-0.017296,-0.016317
5,2020-11-22:21.00.00,1007,12,0.000014,1.178843e-06,-3.472468e-06,-5.514498e-06,327.909836,348.553285,347.429604,...,-0.097499,-0.115552,-0.103627,-0.055830,-0.008271,0.009142,-0.004370,-0.011956,-0.012377,-0.010081
6,2020-11-23:00.00.00,1006,13,0.000014,5.274590e-07,-2.418490e-06,-4.135727e-06,326.838229,348.670526,347.279576,...,-0.096495,-0.110931,-0.098452,-0.071045,-0.031172,-0.010264,-0.008990,-0.002605,0.009843,0.017592
7,2020-11-23:03.00.00,1008,13,0.000013,1.033984e-07,-1.397790e-06,-2.750894e-06,326.928653,348.637400,347.235834,...,-0.084414,-0.093452,-0.087313,-0.070638,-0.038386,-0.018827,-0.012818,-0.005613,0.005607,0.011993
8,2020-11-23:06.00.00,1007,13,0.000013,1.962774e-06,2.047650e-07,-2.571119e-06,327.727050,349.102494,347.594306,...,-0.092110,-0.107825,-0.124465,-0.110838,-0.082611,-0.060366,-0.051719,-0.045475,-0.038422,-0.033897
9,2020-11-23:09.00.00,1004,13,0.000014,6.640247e-07,-2.138302e-06,-2.802736e-06,328.500958,349.547546,347.932249,...,-0.102938,-0.112898,-0.110809,-0.082311,-0.057175,-0.035954,-0.037499,-0.033784,-0.026006,-0.020837


In [107]:
ts_nivar.to_csv(output+'outer_all_nivar.csv')